# Business Problem 


Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.

Buna yönelik olarak müşterilerin davranışlarını tanımlayacağız ve bu davranışlarda öbeklenmelere göre gruplar oluşturacağız.

Yani ortak davranışlar sergileyenleri aynı gruplara alacağız ve bu gruplara özel satış ve pazarlama teknikleri geliştirmeye çalışacağız.

**Veri Seti Hikayesi**

https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.

Bu şirket hediyelik eşya satıyor. Promosyon ürünleri gibi düşünebilir.

Müşterilerinin çoğu da toptancı.

**Değişkenler**

- InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder.
- StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
- Description: Ürün ismi
- Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
- InvoiceDate: Fatura tarihi ve zamanı. 
- UnitPrice: Ürün fiyatı (Sterlin cinsinden)
- CustomerID: Eşsiz müşteri numarası
- Country: Ülke ismi. Müşterinin yaşadığı ülke.



In [67]:
import pandas as pd
import numpy as np
import seaborn as sns

# to display all columns and rows:
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);

#virgulden sonra gösterilecek olan sayı sayısı
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import matplotlib.pyplot as plt 

In [68]:
df_2010_2011 = pd.read_excel("online_retail_II.xlsx", sheet_name = "Year 2010-2011")

In [69]:
df = df_2010_2011.copy()

In [70]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom


In [71]:
#en cok siparis edilen urunlerin sıralaması
df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity", ascending = False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039


In [72]:
#toplam kac fatura kesilmiştir?
df["Invoice"].nunique()

25900

In [73]:
#en pahalı ürünler hangileri?
df.sort_values("Price", ascending = False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
222681,C556445,M,Manual,-1,2011-06-10 15:31:00,38970,15098,United Kingdom
524602,C580605,AMAZONFEE,AMAZON FEE,-1,2011-12-05 11:36:00,17836,nan,United Kingdom
43702,C540117,AMAZONFEE,AMAZON FEE,-1,2011-01-05 09:55:00,16888,nan,United Kingdom
43703,C540118,AMAZONFEE,AMAZON FEE,-1,2011-01-05 09:57:00,16454,nan,United Kingdom
16356,C537651,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:49:00,13541,nan,United Kingdom


In [74]:
#en fazla sipariş sayısına sahip ilk 5 ülke
df["Country"].value_counts().head()

United Kingdom    495478
Germany             9495
France              8558
EIRE                8196
Spain               2533
Name: Country, dtype: int64

In [75]:
#Toplam harcamayı sütun olarak ekledik.
df['TotalPrice'] = df['Price']*df['Quantity']

In [76]:
#hangi ulke ne kadar kazandırdı?
df.groupby("Country").agg({"TotalPrice":"sum"}).sort_values("TotalPrice", ascending = False).head()

,TotalPrice
Country,
United Kingdom,8187806
Netherlands,284662
EIRE,263277
Germany,221698
France,197422


# Data Preparation

In [77]:
df["InvoiceDate"].min()

Timestamp('2010-12-01 08:26:00')

In [78]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [79]:
#bugünün tarihi olarak değerlendirmenin daha kolay yapılabilmesi için 1 Ocak 2012 tarihi belirlendi.  
today = pd.datetime(2012,1,1) 
today

/home/emre/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


datetime.datetime(2012, 1, 1, 0, 0)

In [80]:
today

datetime.datetime(2012, 1, 1, 0, 0)

In [81]:
#Sipariş tarihinin veri tipi değiştirildi
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [82]:
#0'dan büyük değerleri alınması
df = df[df['Quantity'] > 0]
df = df[df['TotalPrice'] > 0]

In [83]:
df.dropna(inplace = True) #eksik verilere sahip gözlem birimlerinin df üzerinden kaldırılması

In [84]:
df.shape

(397885, 9)

In [85]:
df.describe([0.01,0.05,0.10,0.25,0.50,0.75,0.90,0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,397885,13,179,1,1,1,1,2,6,12,24,36,120,80995
Price,397885,3,22,0,0,0,1,1,2,4,6,8,15,8143
Customer ID,397885,15294,1713,12346,12415,12627,12883,13969,15159,16795,17725,17912,18211,18287
TotalPrice,397885,22,309,0,1,1,2,5,12,20,35,68,202,168470


# RFM Skorları ile Müşteri Segmentasyonu

Recency, Frequency, Monetary ifadelerinin baş harflerinden oluşur.

Müşterilerin satın alma alışkanlıkları üzerinden pazarlama ve satış stratejileri belirlemeye yardımcı olan bir tekniktir.

- Recency (yenilik): Müşterinin son satın almasından bugüne kadar geçen süre

    -- Diğer bir ifadesiyle “Müşterinin son temasından bugüne kadar geçen süre” dir.

    -- Bugünün tarihi - Son satın alma

    -- Örnek verecek olursak bugün bu analizi yapıyorsak bugünün tarihi - son 	ürün satın alma tarihi.

    -- Bu örneğin 20 olabilir 100 olabilir. Biliriz ki 20 olan müşteri daha sıcaktır. Daha son zamanlarda bizimle teması olmuştur.

- Frequency (Sıklık): Toplam satın alma sayısı.

- Monetary (Parasal Değer): Müşterinin yaptığı toplam harcama.


In [86]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom,15
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom,22
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom,20


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397885 entries, 0 to 541909
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      397885 non-null  object        
 1   StockCode    397885 non-null  object        
 2   Description  397885 non-null  object        
 3   Quantity     397885 non-null  int64         
 4   InvoiceDate  397885 non-null  datetime64[ns]
 5   Price        397885 non-null  float64       
 6   Customer ID  397885 non-null  float64       
 7   Country      397885 non-null  object        
 8   TotalPrice   397885 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 30.4+ MB


In [88]:
#Recency ve Monetary değerlerinin bulunması.
df_x = df.groupby('Customer ID').agg({'TotalPrice': lambda x: x.sum(), #kişi başına düşen monetary değeri
                                        'InvoiceDate': lambda x: (today - x.max()).days}) #recency değerini buluruz.
# x.max()).days; müşterilerin son alışveriş tarihi

In [89]:
df_y = df.groupby(['Customer ID','Invoice']).agg({'TotalPrice': lambda x: x.sum()})

In [90]:
df_z = df_y.groupby('Customer ID').agg({'TotalPrice': lambda x: len(x)}) 
#kişi başına düşen frequency değerini bulmuş oluruz.

In [91]:
#RFM tablosuna 
rfm_table= pd.merge(df_x,df_z, on='Customer ID')

In [92]:
#Sütun isimlerini belirlenmesi
rfm_table.rename(columns= {'InvoiceDate': 'Recency',
                          'TotalPrice_y': 'Frequency',
                          'TotalPrice_x': 'Monetary'}, inplace= True)

In [93]:
rfm_table.head()

,Monetary,Recency,Frequency
Customer ID,,,
12346,77184,347,1
12347,4310,24,7
12348,1797,97,4
12349,1758,40,1
12350,334,332,1


Tablo Yorumu: Customer ID değeri 12346 olan müşteri 77.184 Sterlin tutarında alışveriş yapmıştır. Bu müşterinin en son alışverişi yaklaşık 1 yıl önce (347 gün) ve yaptığı alışveriş sayısı 1'dir.  

In [94]:
rfm_table['RecencyScore'] = pd.qcut(rfm_table['Recency'],5,labels=[5,4,3,2,1])

In [95]:
rfm_table['FrequencyScore'] = pd.qcut(rfm_table['Frequency'].rank(method="first"),5,labels=[1,2,3,4,5])

In [96]:
rfm_table['MonetaryScore'] = pd.qcut(rfm_table['Monetary'],5,labels=[1,2,3,4,5])

In [97]:
rfm_table.head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore
Customer ID,,,,,,
12346,77184,347,1,1,1,5
12347,4310,24,7,5,5,5
12348,1797,97,4,2,4,4
12349,1758,40,1,4,1,4
12350,334,332,1,1,1,2


In [99]:
(rfm_table['RecencyScore'].astype(str) + 
 rfm_table['FrequencyScore'].astype(str) + 
 rfm_table['MonetaryScore'].astype(str)).head()

Customer ID
12346    115
12347    555
12348    244
12349    414
12350    112
dtype: object

In [102]:
#RFM skorunun hesaplanması
rfm_table["RFM_SCORE"] = rfm_table['RecencyScore'].astype(str) + rfm_table['FrequencyScore'].astype(str) + rfm_table['MonetaryScore'].astype(str)

In [103]:
rfm_table.head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346,77184,347,1,1,1,5,115
12347,4310,24,7,5,5,5,555
12348,1797,97,4,2,4,4,244
12349,1758,40,1,4,1,4,414
12350,334,332,1,1,1,2,112


In [104]:
rfm_table.describe().T

,count,mean,std,min,25%,50%,75%,max
Monetary,4338,2054,8989,4,307,674,1662,280206
Recency,4338,114,100,22,39,72,164,395
Frequency,4338,4,8,1,1,2,5,209


In [105]:
rfm_table[rfm_table["RFM_SCORE"] == "555"].head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12347,4310,24,7,5,5,5,555
12362,5226,25,10,5,5,5,555
12417,3649,25,9,5,5,5,555
12433,13376,22,7,5,5,5,555
12437,4951,23,18,5,5,5,555


In [106]:
rfm_table[rfm_table["RFM_SCORE"] == "111"].head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12353,89,226,1,1,1,1,111
12361,190,309,1,1,1,1,111
12401,84,325,1,1,1,1,111
12402,226,345,1,1,1,1,111
12441,174,388,1,1,1,1,111


In [107]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Lose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [108]:
rfm_table['Segment'] = rfm_table['RecencyScore'].astype(str) + rfm_table['FrequencyScore'].astype(str)
rfm_table['Segment'] = rfm_table['Segment'].replace(seg_map, regex=True)
rfm_table.head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346,77184,347,1,1,1,5,115,Hibernating
12347,4310,24,7,5,5,5,555,Champions
12348,1797,97,4,2,4,4,244,At Risk
12349,1758,40,1,4,1,4,414,Promising
12350,334,332,1,1,1,2,112,Hibernating


In [109]:
rfm_table[["Segment", "Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"])

Recency       Frequency       Monetary      
                       mean count      mean count     mean count
Segment                                                         
About to Sleep           74   352         1   352      472   352
At Risk                 175   593         3   593     1085   593
Can't Lose              154    63         8    63     2796    63
Champions                27   633        12   633     6858   633
Hibernating             239  1071         1  1071      489  1071
Loyal Customers          55   819         6   819     2864   819
Need Attention           73   187         2   187      898   187
New Customers            28    42         1    42      388    42
Potential Loyalists      38   484         2   484     1041   484
Promising                45    94         1    94      294    94

### Yorum: 

Değerlendirme amacıyla seçilen 3 segment Can't Lose, Hibernating ve Champions segmentleridir.

Segmentlere ait müşteri sayısı:
Can't Loose = 63, Hibernating = 1071, Champions = 633


- Can't Lose Segmenti;

    - Müşterilerin en son yaptıkları alışveriş tarihi ortalama 154 gün öncedir.
    - Müşteriler ortalama 8 kez alışveriş yapmışlardır.
    - Müşteriler ortalama 2796 Sterlin değerinde alışveriş yapmışlardır.


- Hibernating Segmenti;

    - Müşterilerin en son yaptıkları alışveriş tarihi ortalama 239 gün öncedir.
    - Müşteriler ortalama 1 kez alışveriş yapmışlardır.
    - Müşteriler ortalama 489 Sterlin değerinde alışveriş yapmışlardır.


- Champions Segmenti;

    - Müşterilerin en son yaptıkları alışveriş tarihi ortalama 27 gün öncedir.
    - Müşteriler ortalama 12 kez alışveriş yapmışlardır.
    - Müşteriler ortalama 6858 Sterlin değerinde alışveriş yapmışlardır.


Can't Lose Segmenti;

Bu segmentteki müşteriler son zamanlarda bir satın alma işlemi gerçekleştirmediler. Bu sebeple bu segment için bir indirim ve hediye kampanyası hazırlamamız gerekir. Bu müşteriler önceden alışveriş yaptıklarında fazla sayıda alışveriş yapmışlardır fakat recency değerleri olması gereken değerden düşük durumdadır. Bu müşteriler için uygulanacak kampanya, hem satın alınan öğeler hem de görüntülenen öğelerden oluşan önceki etkinliklerine dayanan öneriler içermelidir. Daha önce ilgilendikleri ürünlerle ilişkili olan yeni ve popüler ürünler de bu kampanya içerisinde yer alabilir. Bu müşterilerin satın almaktan vazgeçmiş olmalarına sebep olacak durumların araştırılması gerekir. 


Hibernating Segmenti;

Bu segmentteki müşteriler uzun bir süre boyunca bir satın alma işlemi gerçekleştirmemişlerdir. Fakat indirimler sunularak başka bir satın alma yapmaları için ilgileri yeniden çekilebilir.


Champions Segmenti;

Bu segmentteki müşteriler gelirin büyük bir kısmından sorumludur. Bu müşterilerin alışverişlerinin devamını sağlayacak kampanyalar uygulanmalıdır.

### "Need Attention" sınıfına ait customer ID'lerin seçilerek excel çıktısının alınması.

In [110]:
rfm_table[rfm_table["Segment"] == "Need Attention"].index

Float64Index([12360.0, 12372.0, 12413.0, 12456.0, 12536.0, 12577.0, 12759.0,
              12808.0, 12879.0, 12885.0,
              ...
              17928.0, 17934.0, 17999.0, 18147.0, 18160.0, 18205.0, 18228.0,
              18252.0, 18261.0, 18270.0],
             dtype='float64', name='Customer ID', length=187)

In [111]:
new_df = pd.DataFrame()
new_df["Need Attention ID"] = rfm_table[rfm_table["Segment"] == "Need Attention"].index

In [112]:
new_df.head()

,Need Attention ID
0,12360
1,12372
2,12413
3,12456
4,12536


In [113]:
new_df.to_excel("need_attention.xlsx")